In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
MAR_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/MAR_predictions_SVM.csv", index_col=0)


In [3]:
MAR_predictions.head()

,Date,Predictions
0,2019-07-31,0
1,2019-06-28,0
2,2019-05-31,1
3,2019-04-30,1
4,2019-03-29,0


In [4]:
MAR_predictions.Date = pd.to_datetime(MAR_predictions.Date, format="%Y-%m-%d")

In [5]:
MAR_predictions['Predictions'] = MAR_predictions['Predictions'].shift(-1)


In [6]:
MAR_predictions = MAR_predictions.dropna()


In [7]:
MAR_predictions = MAR_predictions[::-1]

In [8]:
MAR_predictions.head(10)

,Date,Predictions
112,2010-03-31,1.0
111,2010-04-30,0.0
110,2010-05-28,0.0
109,2010-06-30,0.0
108,2010-07-30,1.0
107,2010-08-31,1.0
106,2010-09-30,1.0
105,2010-10-29,0.0
104,2010-11-30,1.0
103,2010-12-31,0.0


In [9]:
MAR_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/MAR/MAR_monthly.csv", index_col =0)

In [10]:
MAR_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
MAR_predictions = MAR_predictions.set_index("Date")

In [12]:
MAR_monthly.head()

,Open,High,Low,Close,Volume
2019-08-15,138.99,140.490,124.77,126.70,23829100
2019-07-31,142.41,144.240,135.39,139.06,31330900
2019-06-28,124.76,140.460,123.46,140.29,31391900
2019-05-31,137.80,140.371,122.46,124.84,40716900
2019-04-30,126.24,137.100,125.85,136.42,29052100


In [13]:
MAR_final_df = pd.merge(MAR_predictions, MAR_monthly, right_index=True, left_index=True)

In [14]:
MAR_final_df.index = pd.to_datetime(MAR_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0 and (position.is_short!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [17]:
bt = Backtest(MAR_final_df, Binary_Classification_1d, cash=10000, commission=.01)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-07-31 00:00:00
Duration                        3409 days 00:00:00
Exposure [%]                               97.3306
Equity Final [$]                            237476
Equity Peak [$]                             239577
Return [%]                                 2274.76
Buy & Hold Return [%]                       341.18
Max. Drawdown [%]                         -17.8982
Avg. Drawdown [%]                         -5.15239
Max. Drawdown Duration           274 days 00:00:00
Avg. Drawdown Duration            96 days 00:00:00
# Trades                                        51
Win Rate [%]                               70.5882
Best Trade [%]                             49.0394
Worst Trade [%]                           -15.2053
Avg. Trade [%]                             6.36772
Max. Trade Duration              213 days 00:00:00
Avg. Trade Duration               66 days 00:00:00
Expectancy [%]                 

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/MAR_SVM_Equity.csv")

In [23]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,10855.785741
2010-06-30,11794.534397
2010-07-30,10732.758851
2010-08-31,9683.521415
2010-09-30,10850.316550
2010-10-29,11219.766346
2010-11-30,10646.942263
2010-12-31,10709.587025
2011-01-31,11310.625118
